<h1><center>Pawpular EDA</center></h1>

## Contents
1. Introductory remarks 
2. Tabular Data Analysis
3. Image Data Analysis

## Introductory remarks
It is mentioned in the problem statement that the **tabular data is not used to calculate the popularity**(Dependent variable) of the image, but we will initially go ahead and perform an EDA on the tabular data to understand the distribution of features in the images and also how does the presence of those featues impact them image popularity.

## Tabular Data Analysis

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px

import plotly.graph_objs as go
import matplotlib.pyplot as plt

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
from plotly import tools
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
        #   print(os.path.join(dirname, filename))

In [ ]:
df_train= pd.read_csv("/kaggle/input/petfinder-pawpularity-score/train.csv")


In [ ]:
df_categorical = df_train.iloc[:,1:13]  
cat_cols=df_categorical.columns
df_categorical=df_categorical.apply(pd.value_counts).fillna(0)
df_categorical=df_categorical.transpose()
df_categorical["Attribute"]=df_categorical.index
df_categorical.columns=["Absent","Present","Attribute"]

fig = px.bar(df_categorical, x="Attribute", y=["Absent","Present"],
             barmode='group',color_discrete_sequence=["red", "green"],
             height=400)
          
fig.update_layout(
    title="Image Count across Attribute Presence",
    xaxis_title="Attribute",
    yaxis_title="Count",
legend_title="Presence")
fig.show()

In [ ]:
df_avg_all=pd.DataFrame()
for i in cat_cols:
    df_avg=df_train[[i,"Pawpularity"]]
    df_avg=df_avg.groupby([i],as_index=False)['Pawpularity'].agg(np.mean)
    df_avg[i]=df_avg["Pawpularity"]
    df_avg_all[i]=df_avg[i]

df_avg_all=df_avg_all.transpose()
df_avg_all["Attribute"]=df_avg_all.index
df_avg_all.columns=["Absent","Present","Attribute"]

fig = px.line(df_avg_all, x="Attribute", y=["Absent","Present"]
           ,color_discrete_sequence=["red", "green"],
             height=400)      
fig.update_layout(
    title="Pawpularity Average across Attribute Presence",
    xaxis_title="Attribute",
    yaxis_title="Pawpularity Average",
legend_title="Presence")
fig.show()

Findings from the above chart:
* The presence of accessories and groups leads to higher engagement of the image
* The absense of Blur results in better engagement

## Image Data Analysis

In [ ]:
df_ordered=df_train.sort_values(by="Pawpularity").reset_index(drop=True)
df_lowest_popularity=df_ordered.head(5)
df_highest_popularity=df_ordered.tail(5).reset_index(drop=True)


In [ ]:
#Viewing first five some sample images and their corresponding poppularity and shape
print("Images with lowest popularity:")
j=1
plt.figure(figsize=(20,20)) 
for i in df_lowest_popularity["Id"]:
    img_loc= "/kaggle/input/petfinder-pawpularity-score/train/"+i+".jpg"

    cv_img = cv2.imread(img_loc)
    cv_img=cv2.cvtColor(cv_img,cv2.COLOR_BGR2RGB)
    Original_Image_Size="Original Size:"+str(cv_img.shape)
    cv_img = cv2.resize(cv_img, (500, 500))
    plt.subplot(1,5,j)    # the number of images in the grid is 5*5 (25)
    plt.imshow(cv_img)
    plt.title(Original_Image_Size)
    plt.xlabel("Popularity:" +str(df_lowest_popularity["Pawpularity"][j-1]))
    plt.xticks([])
    plt.yticks([])

    j=j+1
    
plt.show()

In [ ]:
#Viewing first five some sample images and their corresponding poppularity and shape
print("Images with highest popularity:")
j=1
plt.figure(figsize=(20,20)) 
for i in df_highest_popularity["Id"]:
    img_loc= "/kaggle/input/petfinder-pawpularity-score/train/"+i+".jpg"

    cv_img = cv2.imread(img_loc)
    cv_img=cv2.cvtColor(cv_img,cv2.COLOR_BGR2RGB)
    Original_Image_Size="Original Size:"+str(cv_img.shape)
    cv_img = cv2.resize(cv_img, (500, 500))
    plt.subplot(1,5,j)    # the number of images in the grid is 5*5 (25)
    plt.imshow(cv_img)
    plt.title(Original_Image_Size)
    plt.xlabel("Popularity:" +str(df_highest_popularity["Pawpularity"][j-1]))
    plt.xticks([])
    plt.yticks([])

    j=j+1
    
plt.show()

## Create File dataframe

In [ ]:
df_file_train=df_train[["Id","Pawpularity"]]
df_file_train["File_loc"]="/kaggle/input/petfinder-pawpularity-score/train/"+ df_file_train["Id"]+".jpg"

In [ ]:

image_df = df_file_train.sample(9912,random_state=1).reset_index(drop=True)
train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)


In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='File_loc',
    y_col='Pawpularity',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='File_loc',
    y_col='Pawpularity',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)


test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='File_loc',
    y_col='Pawpularity',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

In [ ]:
inputs = tf.keras.Input(shape=(120, 120, 3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

In [ ]:
predicted_ages = np.squeeze(model.predict(test_images))
true_ages = test_images.labels

rmse = np.sqrt(model.evaluate(test_images, verbose=0))
print("     Test RMSE: {:.5f}".format(rmse))

## Predictions for submission

In [ ]:
pred_sub= pd.read_csv("/kaggle/input/petfinder-pawpularity-score/test.csv")
pred_sub=pred_sub[["Id"]]
pred_sub["File_loc"]="/kaggle/input/petfinder-pawpularity-score/test/"+ pred_sub["Id"]+".jpg"
pred_sub["Pawpularity"]=1#creating dummy target variable

In [ ]:
pred_images = test_generator.flow_from_dataframe(
    dataframe=pred_sub,
    x_col='File_loc',
    y_col='Pawpularity',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

In [ ]:
predicted_pawpularity = np.squeeze(model.predict(pred_images))
predicted_pawpularity

In [ ]:
df_ss=pd.DataFrame({'Id':pred_sub["Id"],
        'Pawpularity':predicted_pawpularity})
df_ss.to_csv("submission.csv",index=False)                             